In [52]:
import pandas as pd
import numpy as np
import sklearn
import os
from tqdm import tqdm, trange, tqdm_notebook
import re
import nltk
from multiprocessing import cpu_count, Pool
from functools import partial
import ipywidgets
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from rewrite.scorer import score_4class
import utils # utils from CS224U
from scipy.spatial import distance
import random
tqdm.pandas()

In [53]:
X_train = pd.read_hdf("X_train_full_allfeatures-NOLABEL.h5", key="df")
y_train = pd.read_hdf("y_train_full.h5", key="df")

In [54]:
y_train.head()

0    unrelated
1        agree
2    unrelated
3    unrelated
4     disagree
Name: Stance, dtype: object

In [55]:
X_train.head()

,Headline,articleBody,___clean_headline,___clean_headline_tokenized_lemmas,___clean_body,___clean_body_tokenized_lemmas,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,...,chargram_2_first_hits,chargram_8_hits,chargram_8_early_hits,chargram_8_first_hits,chargram_4_hits,chargram_4_early_hits,chargram_4_first_hits,chargram_16_hits,chargram_16_early_hits,chargram_16_first_hits
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,police find mass graves with at least 15 bodie...,"[police, find, mass, graf, with, at, least, 15...",danny boyle is directing the untitled film set...,"[danny, boyle, is, directing, the, untitled, f...",2,0,0,0.014085,...,0,0,0,0,0,0,0,0,0,0
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,hundreds of palestinians flee floods in gaza a...,"[hundred, of, palestinian, flee, flood, in, ga...",hundreds of palestinians were evacuated from t...,"[hundred, of, palestinian, were, evacuated, fr...",10,7,7,0.046083,...,3,0,0,0,0,0,0,0,0,0
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,christian bale passes on role of steve jobs ac...,"[christian, bale, pass, on, role, of, steve, j...",30 year old moscow resident was hospitalized w...,"[30, year, old, moscow, resident, wa, hospital...",5,4,1,0.030303,...,1,0,0,0,0,0,0,0,0,0
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,hbo and apple in talks for 15 month apple tv s...,"[hbo, and, apple, in, talk, for, 15, month, ap...",reuters a canadian soldier was shot at the can...,"[reuters, a, canadian, soldier, wa, shot, at, ...",3,3,0,0.028169,...,1,0,0,0,0,0,0,0,0,0
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",spider burrowed through tourist s stomach and ...,"[spider, burrowed, through, tourist, s, stomac...",fear not arachnophobes the story of bunbury s ...,"[fear, not, arachnophobes, the, story, of, bun...",9,5,4,0.032727,...,4,0,0,0,0,0,0,0,0,0


In [56]:
print(X_train.shape)
print(y_train.shape)

(49972, 49)
(49972,)


In [57]:
def del_str_cols(df): # df should be X, e.g. X_train or X_dev
    del df["articleBody"]
    del df["Headline"]
    for col_name in df.columns:
        if "___" == col_name[0:3]:
            del df[col_name]
#del_str_cols()

In [58]:
Xy_train = pd.concat([X_train, y_train], axis=1)
print(Xy_train.shape)

(49972, 50)


In [59]:
Xy_train.head()

,Headline,articleBody,___clean_headline,___clean_headline_tokenized_lemmas,___clean_body,___clean_body_tokenized_lemmas,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,...,chargram_8_hits,chargram_8_early_hits,chargram_8_first_hits,chargram_4_hits,chargram_4_early_hits,chargram_4_first_hits,chargram_16_hits,chargram_16_early_hits,chargram_16_first_hits,Stance
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,police find mass graves with at least 15 bodie...,"[police, find, mass, graf, with, at, least, 15...",danny boyle is directing the untitled film set...,"[danny, boyle, is, directing, the, untitled, f...",2,0,0,0.014085,...,0,0,0,0,0,0,0,0,0,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,hundreds of palestinians flee floods in gaza a...,"[hundred, of, palestinian, flee, flood, in, ga...",hundreds of palestinians were evacuated from t...,"[hundred, of, palestinian, were, evacuated, fr...",10,7,7,0.046083,...,0,0,0,0,0,0,0,0,0,agree
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,christian bale passes on role of steve jobs ac...,"[christian, bale, pass, on, role, of, steve, j...",30 year old moscow resident was hospitalized w...,"[30, year, old, moscow, resident, wa, hospital...",5,4,1,0.030303,...,0,0,0,0,0,0,0,0,0,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,hbo and apple in talks for 15 month apple tv s...,"[hbo, and, apple, in, talk, for, 15, month, ap...",reuters a canadian soldier was shot at the can...,"[reuters, a, canadian, soldier, wa, shot, at, ...",3,3,0,0.028169,...,0,0,0,0,0,0,0,0,0,unrelated
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",spider burrowed through tourist s stomach and ...,"[spider, burrowed, through, tourist, s, stomac...",fear not arachnophobes the story of bunbury s ...,"[fear, not, arachnophobes, the, story, of, bun...",9,5,4,0.032727,...,0,0,0,0,0,0,0,0,0,disagree


In [60]:
#Xy_train = Xy_train[Xy_train["Stance"] != "unrelated"]

In [61]:
print(Xy_train.shape)

(49972, 50)


In [62]:
y_train = Xy_train["Stance"]
X_train = Xy_train.drop("Stance", axis=1)
print(X_train.shape)
print(y_train.shape)

(49972, 49)
(49972,)


In [63]:
# START SENTIMENT ANALYSIS

In [64]:
tqdm.pandas()

In [65]:
from nltk.sentiment import vader
sid = vader.SentimentIntensityAnalyzer()

In [66]:
def vader_polarity_scores(df, text_col_name, col_name_prefix):
    pol_scores = df[text_col_name].progress_apply(lambda hl: pd.Series(sid.polarity_scores(hl)))
    cols = pol_scores.columns
    new_cols = []
    for col_name in cols:
        new_cols.append("vader_"+col_name_prefix+"_"+col_name)
    pol_scores.columns = new_cols
    return pol_scores

vader_hl_df = vader_polarity_scores(X_train, "Headline", "hl")
vader_body_df = vader_polarity_scores(X_train, "articleBody", "body")
X_train = pd.concat([X_train, vader_hl_df, vader_body_df], axis=1)













  0%|          | 0/49972 [00:00<?, ?it/s]











  0%|          | 159/49972 [00:00<00:31, 1587.60it/s]











  1%|          | 281/49972 [00:00<00:34, 1455.70it/s]











  1%|          | 463/49972 [00:00<00:31, 1547.38it/s]











  1%|▏         | 640/49972 [00:00<00:30, 1606.51it/s]











  2%|▏         | 826/49972 [00:00<00:29, 1673.30it/s]











  2%|▏         | 994/49972 [00:00<00:29, 1673.81it/s]











  2%|▏         | 1145/49972 [00:00<00:34, 1397.96it/s]











  3%|▎         | 1295/49972 [00:00<00:34, 1426.95it/s]











  3%|▎         | 1457/49972 [00:00<00:32, 1478.50it/s]











  3%|▎         | 1604/49972 [00:01<00:35, 1377.85it/s]











  3%|▎         | 1748/49972 [00:01<00:34, 1393.70it/s]











  4%|▍         | 1888/49972 [00:01<00:35, 1363.89it/s]











  4%|▍         | 2033/49972 [00:01<00:34, 1387.42it/s]











  4%|▍         | 2205/49972 [00:01<00:32, 1471.67it/s]











  5%|▍         | 

 35%|███▌      | 17696/49972 [00:17<00:13, 2363.15it/s]











 36%|███▌      | 17959/49972 [00:17<00:13, 2436.30it/s]











 36%|███▋      | 18213/49972 [00:18<00:15, 2105.21it/s]











 37%|███▋      | 18502/49972 [00:18<00:13, 2291.67it/s]











 38%|███▊      | 18759/49972 [00:18<00:13, 2366.99it/s]











 38%|███▊      | 19038/49972 [00:18<00:12, 2477.37it/s]











 39%|███▊      | 19329/49972 [00:18<00:11, 2591.84it/s]











 39%|███▉      | 19622/49972 [00:18<00:11, 2684.79it/s]











 40%|███▉      | 19899/49972 [00:18<00:11, 2707.25it/s]











 40%|████      | 20102/49972 [00:29<08:10, 60.94it/s]  











 40%|████      | 20102/49972 [00:29<08:10, 60.94it/s]











 41%|████      | 20251/49972 [00:29<05:47, 85.56it/s]











 41%|████      | 20482/49972 [00:29<04:05, 120.31it/s]











 41%|████▏     | 20720/49972 [00:29<02:53, 168.22it/s]











 42%|████▏     | 20915/49972 [00:29<02:05, 231.73it/s]











 4

 76%|███████▌  | 37863/49972 [00:39<00:05, 2346.16it/s]











 76%|███████▋  | 38125/49972 [00:39<00:04, 2374.44it/s]











 77%|███████▋  | 38388/49972 [00:39<00:04, 2445.53it/s]











 77%|███████▋  | 38652/49972 [00:39<00:04, 2500.04it/s]











 78%|███████▊  | 38912/49972 [00:39<00:04, 2459.76it/s]











 78%|███████▊  | 39166/49972 [00:39<00:04, 2296.96it/s]











 79%|███████▉  | 39403/49972 [00:39<00:04, 2201.72it/s]











 79%|███████▉  | 39671/49972 [00:40<00:04, 2325.11it/s]











 80%|███████▉  | 39942/49972 [00:40<00:04, 2428.17it/s]











 80%|████████  | 40198/49972 [00:40<00:03, 2462.70it/s]











 81%|████████  | 40465/49972 [00:40<00:03, 2518.92it/s]











 81%|████████▏ | 40720/49972 [00:40<00:05, 1827.34it/s]











 82%|████████▏ | 40932/49972 [00:40<00:06, 1316.57it/s]











 82%|████████▏ | 41105/49972 [00:41<00:09, 924.36it/s] 











 83%|████████▎ | 41253/49972 [00:41<00:08, 1041.58it/s]








  1%|          | 497/49972 [00:03<05:07, 160.77it/s]











  1%|          | 522/49972 [00:03<04:34, 179.90it/s]











  1%|          | 544/49972 [00:03<04:20, 189.38it/s]











  1%|          | 564/49972 [00:03<04:45, 172.87it/s]











  1%|          | 587/49972 [00:03<04:27, 184.94it/s]











  1%|          | 607/49972 [00:03<04:24, 186.97it/s]











  1%|▏         | 627/49972 [00:03<04:37, 178.05it/s]











  1%|▏         | 649/49972 [00:03<04:21, 188.61it/s]











  1%|▏         | 679/49972 [00:03<03:52, 211.74it/s]











  1%|▏         | 702/49972 [00:04<04:00, 204.77it/s]











  1%|▏         | 724/49972 [00:04<04:32, 180.72it/s]











  1%|▏         | 748/49972 [00:04<04:14, 193.70it/s]











  2%|▏         | 769/49972 [00:04<04:09, 197.22it/s]











  2%|▏         | 790/49972 [00:04<04:41, 174.84it/s]











  2%|▏         | 809/49972 [00:04<04:36, 178.02it/s]











  2%|▏         | 828/49972 [00:04<04:36,

  4%|▍         | 2215/49972 [00:12<04:37, 171.79it/s]











  4%|▍         | 2233/49972 [00:12<04:35, 173.11it/s]











  5%|▍         | 2254/49972 [00:12<04:23, 181.18it/s]











  5%|▍         | 2275/49972 [00:12<04:15, 186.68it/s]











  5%|▍         | 2295/49972 [00:12<04:17, 185.08it/s]











  5%|▍         | 2314/49972 [00:12<04:27, 178.13it/s]











  5%|▍         | 2339/49972 [00:12<04:04, 194.50it/s]











  5%|▍         | 2360/49972 [00:13<05:58, 132.72it/s]











  5%|▍         | 2377/49972 [00:13<06:24, 123.84it/s]











  5%|▍         | 2392/49972 [00:13<06:56, 114.13it/s]











  5%|▍         | 2406/49972 [00:13<07:36, 104.24it/s]











  5%|▍         | 2420/49972 [00:13<07:02, 112.49it/s]











  5%|▍         | 2437/49972 [00:13<06:23, 123.85it/s]











  5%|▍         | 2453/49972 [00:13<06:10, 128.39it/s]











  5%|▍         | 2467/49972 [00:14<08:10, 96.83it/s] 











  5%|▍         | 2480/499

...

 87%|████████▋ | 43692/49972 [04:49<00:31, 200.84it/s]











 87%|████████▋ | 43716/49972 [04:49<00:29, 209.15it/s]











 88%|████████▊ | 43738/49972 [04:49<00:29, 209.99it/s]











 88%|████████▊ | 43760/49972 [04:49<00:31, 199.64it/s]











 88%|████████▊ | 43781/49972 [04:49<00:32, 191.02it/s]











 88%|████████▊ | 43806/49972 [04:49<00:30, 203.45it/s]











 88%|████████▊ | 43828/49972 [04:50<00:29, 206.21it/s]











 88%|████████▊ | 43850/49972 [04:50<00:29, 209.43it/s]











 88%|████████▊ | 43877/49972 [04:50<00:28, 217.06it/s]











 88%|████████▊ | 43899/49972 [04:50<00:28, 212.42it/s]











 88%|████████▊ | 43921/49972 [04:50<00:28, 209.47it/s]











 88%|████████▊ | 43944/49972 [04:50<00:28, 214.73it/s]











 88%|████████▊ | 43966/49972 [04:50<00:35, 169.53it/s]











 88%|████████▊ | 43988/49972 [04:50<00:33, 177.71it/s]











 88%|████████▊ | 44008/49972 [04:51<00:32, 181.72it/s]











 88%|█████

 91%|█████████ | 45529/49972 [04:58<00:22, 196.67it/s]











 91%|█████████ | 45550/49972 [04:58<00:22, 199.48it/s]











 91%|█████████ | 45571/49972 [04:58<00:21, 200.13it/s]











 91%|█████████ | 45598/49972 [04:58<00:20, 215.62it/s]











 91%|█████████▏| 45601/49972 [05:11<1:28:39,  1.22s/it]











 91%|█████████▏| 45601/49972 [05:11<1:28:39,  1.22s/it]











 91%|█████████▏| 45618/49972 [05:11<1:01:57,  1.17it/s]











 91%|█████████▏| 45642/49972 [05:11<43:13,  1.67it/s]  











 91%|█████████▏| 45662/49972 [05:11<30:13,  2.38it/s]











 91%|█████████▏| 45689/49972 [05:11<21:06,  3.38it/s]











 91%|█████████▏| 45708/49972 [05:11<14:49,  4.79it/s]











 92%|█████████▏| 45733/49972 [05:11<10:24,  6.79it/s]











 92%|█████████▏| 45756/49972 [05:11<07:20,  9.58it/s]











 92%|█████████▏| 45777/49972 [05:11<05:16, 13.25it/s]











 92%|█████████▏| 45796/49972 [05:12<03:48, 18.26it/s]











 92%|████████

 93%|█████████▎| 46705/49972 [05:21<00:39, 82.15it/s]











 93%|█████████▎| 46716/49972 [05:21<00:37, 87.32it/s]











 94%|█████████▎| 46726/49972 [05:21<00:38, 84.40it/s]











 94%|█████████▎| 46736/49972 [05:22<00:39, 81.61it/s]











 94%|█████████▎| 46761/49972 [05:22<00:31, 101.78it/s]











 94%|█████████▎| 46780/49972 [05:22<00:27, 116.42it/s]











 94%|█████████▎| 46795/49972 [05:22<00:25, 122.23it/s]











 94%|█████████▎| 46813/49972 [05:22<00:23, 133.62it/s]











 94%|█████████▎| 46829/49972 [05:22<00:23, 131.15it/s]











 94%|█████████▎| 46847/49972 [05:22<00:23, 135.63it/s]











 94%|█████████▍| 46862/49972 [05:22<00:23, 134.12it/s]











 94%|█████████▍| 46878/49972 [05:23<00:22, 140.56it/s]











 94%|█████████▍| 46893/49972 [05:23<00:25, 119.22it/s]











 94%|█████████▍| 46908/49972 [05:23<00:24, 123.50it/s]











 94%|█████████▍| 46922/49972 [05:23<00:28, 107.34it/s]











 94%|█████████

 96%|█████████▌| 48080/49972 [05:31<00:12, 156.82it/s]











 96%|█████████▌| 48098/49972 [05:31<00:12, 152.10it/s]











 96%|█████████▋| 48115/49972 [05:31<00:13, 141.87it/s]











 96%|█████████▋| 48136/49972 [05:31<00:11, 157.01it/s]











 96%|█████████▋| 48154/49972 [05:31<00:11, 160.46it/s]











 96%|█████████▋| 48177/49972 [05:31<00:10, 175.62it/s]











 96%|█████████▋| 48196/49972 [05:31<00:10, 170.17it/s]











 96%|█████████▋| 48214/49972 [05:32<00:10, 165.22it/s]











 97%|█████████▋| 48232/49972 [05:32<00:11, 152.17it/s]











 97%|█████████▋| 48253/49972 [05:32<00:10, 165.36it/s]











 97%|█████████▋| 48271/49972 [05:32<00:10, 158.20it/s]











 97%|█████████▋| 48289/49972 [05:32<00:10, 161.23it/s]











 97%|█████████▋| 48306/49972 [05:32<00:11, 147.32it/s]











 97%|█████████▋| 48322/49972 [05:32<00:11, 147.40it/s]











 97%|█████████▋| 48342/49972 [05:32<00:10, 154.67it/s]











 97%|█████

 99%|█████████▉| 49505/49972 [05:40<00:02, 160.64it/s]











 99%|█████████▉| 49523/49972 [05:40<00:02, 164.51it/s]











 99%|█████████▉| 49540/49972 [05:40<00:02, 162.95it/s]











 99%|█████████▉| 49560/49972 [05:41<00:02, 171.36it/s]











 99%|█████████▉| 49579/49972 [05:41<00:02, 175.70it/s]











 99%|█████████▉| 49597/49972 [05:41<00:02, 176.66it/s]











 99%|█████████▉| 49615/49972 [05:41<00:02, 163.11it/s]











 99%|█████████▉| 49632/49972 [05:41<00:02, 159.61it/s]











 99%|█████████▉| 49651/49972 [05:41<00:01, 164.94it/s]











 99%|█████████▉| 49672/49972 [05:41<00:01, 175.42it/s]











 99%|█████████▉| 49690/49972 [05:41<00:01, 176.60it/s]











 99%|█████████▉| 49709/49972 [05:41<00:01, 179.89it/s]











100%|█████████▉| 49728/49972 [05:41<00:01, 171.80it/s]











100%|█████████▉| 49746/49972 [05:42<00:01, 166.77it/s]











100%|█████████▉| 49763/49972 [05:42<00:01, 142.54it/s]











100%|█████

In [67]:
# END SENTIMENT ANALYSIS

In [68]:
# START GLOVE AND POS TAGGING

In [69]:
### GLOVE ####
glove_dim = 200
glove_src = os.path.join("GloVe", 'glove.6B.'+str(glove_dim)+'d.txt')
GLOVE = utils.glove2dict(glove_src)
def text_to_mean_vec_ignore_unk(text, w2v=GLOVE, dim=glove_dim):
    vec = np.zeros(dim)
    num_added = 0
    for word in text:
        if word in w2v:
            vec += w2v[word]
            num_added += 1
    if num_added > 0:
        return vec/num_added
    else:
        return np.array([random.uniform(-0.5, 0.5) for i in range(glove_dim)])
def get_glove_cos_dist_hl_body(row):
    hl = row["___clean_headline_tokenized_lemmas"]
    body = row["___clean_body_tokenized_lemmas"]
    hl_vec = text_to_mean_vec_ignore_unk(hl)
    body_vec = text_to_mean_vec_ignore_unk(body)
    cosine_dist = distance.cosine(hl_vec, body_vec) # cosine() from scipy
    return cosine_dist
### END GLOVE CODE BLOCK
X_train["hl_body_glove_"+str(glove_dim)+"_cos_dist"] = X_train[["___clean_headline_tokenized_lemmas", "___clean_body_tokenized_lemmas"]].progress_apply(get_glove_cos_dist_hl_body, axis=1)


nltk.download('averaged_perceptron_tagger')

def get_verbs(text):
    text = nltk.tokenize.word_tokenize(text)
    verbs = [token for token, pos in nltk.pos_tag(text) if pos.startswith('VB')]
    verbs_sentence = ' '.join(word[0] for word in verbs)
    return verbs_sentence
 
def get_verb_glove_cos_dist_hl_body(row):
    hl = row["___clean_headline_tokenized_lemmas"]
    body = row["___clean_body_tokenized_lemmas"]
    
    h1_verbs = get_verbs(h1)
    body_verbs = get_verbs(body)
    
    hl_vec = text_to_mean_vec_ignore_unk(h1_verbs)
    body_vec = text_to_mean_vec_ignore_unk(body_verbs)
    cosine_dist = distance.cosine(hl_vec, body_vec) # cosine() from scipy
    return cosine_dist

X_train["hl_body_verb_glove_"+str(glove_dim)+"_cos_dist"] = X_train[["___clean_headline_tokenized_lemmas", "___clean_body_tokenized_lemmas"]].progress_apply(get_verb_glove_cos_dist_hl_body, axis=1)


AttributeError: module 'utils' has no attribute 'glove2dict'

In [ ]:
# END GLOVE AND  POS TAGGING

In [ ]:
del_str_cols(X_train)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
### LOAD TEST SET ###

In [ ]:
X_test = pd.read_hdf("X_test_full_allfeatures-NOLABEL.h5", key="df")
y_test = pd.read_hdf("y_test_full.h5", key="df")

Xy_test = pd.concat([X_test, y_test], axis=1)

y_test = Xy_test["Stance"]
X_test = Xy_test.drop("Stance", axis=1)

vader_hl_df = vader_polarity_scores(X_test, "Headline", "hl")
vader_body_df = vader_polarity_scores(X_test, "articleBody", "body")
X_test = pd.concat([X_test, vader_hl_df, vader_body_df], axis=1)

X_test["hl_body_glove_"+str(glove_dim)+"_cos_dist"] = X_test[["___clean_headline_tokenized_lemmas", "___clean_body_tokenized_lemmas"]].progress_apply(get_glove_cos_dist_hl_body, axis=1)
X_test["hl_body_verb_glove_"+str(glove_dim)+"_cos_dist"] = X_test[["___clean_headline_tokenized_lemmas", "___clean_body_tokenized_lemmas"]].progress_apply(get_verb_glove_cos_dist_hl_body, axis=1)


del_str_cols(X_test)

In [ ]:
X_train, X_dev, y_train, y_dev = sklearn.model_selection.train_test_split(X_train, y_train, test_size=0.25, random_state=42, shuffle=True)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_dev.shape)
print(y_dev.shape)

In [ ]:
X_train.head()

In [ ]:
clf = GradientBoostingClassifier(n_estimators=200, random_state=14128, verbose=True)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_dev)

In [ ]:
print(classification_report(y_dev, preds))
# With VADER sentiment features

In [ ]:
score, max_score = score_4class(y_dev, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

In [ ]:
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

score, max_score = score_4class(y_test, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

In [ ]:
preds = clf.predict(X_train)
print(classification_report(y_train, preds))

score, max_score = score_4class(y_train, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

In [ ]:
crf = sklearn.ensemble.RandomForestClassifier(n_estimators=400, n_jobs=8, random_state=42)
crf.fit(X_train, y_train)

In [ ]:
preds = crf.predict(X_dev)
print(classification_report(y_dev, preds))

In [ ]:
score, max_score = score_4class(y_dev, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

In [ ]:
preds = crf.predict(X_test)
print(classification_report(y_test, preds))

score, max_score = score_4class(y_test, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

In [ ]:
preds = crf.predict(X_train)
print(classification_report(y_train, preds))

score, max_score = score_4class(y_train, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

In [ ]:
clf.fit(X_test, y_test)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

preds = clf.predict(X_dev)
print(classification_report(y_dev, preds))

In [ ]:
score, max_score = score_4class(y_dev, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

In [ ]:
feat_imp = clf.feature_importances_
indices = np.argsort(feat_imp)[::-1]
for ii in indices:
    print(X_train.columns[ii]+": "+str(feat_imp[ii]))

In [ ]:
-----

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
crf = sklearn.ensemble.RandomForestClassifier(n_estimators=200, n_jobs=8, random_state=42)

In [ ]:
crf.fit(X_train, y_train)

In [ ]:
preds = crf.predict(X_dev)
print(classification_report(y_dev, preds))

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
# 4-class problem:
# By guessing most common category (unrelated), we would achieve approx. 73% accuracy. The baseline model achieves 88%.

In [ ]:
### 3 class problem filter out unrelated

In [ ]:
Xy_train = pd.concat([X_train, y_train], axis=1)
print(Xy_train.shape)

In [ ]:
Xy_train = Xy_train[Xy_train["Stance"] != "unrelated"]
print(Xy_train.shape)

In [ ]:
y_train = Xy_train["Stance"]
X_train = Xy_train.drop("Stance", axis=1)
print(X_train.shape)
print(y_train.shape)

In [ ]:
Xy_dev = pd.concat([X_dev, y_dev], axis=1)
Xy_dev = Xy_dev[Xy_dev["Stance"] != "unrelated"]
y_dev = Xy_dev["Stance"]
X_dev = Xy_dev.drop("Stance", axis=1)
print(X_dev.shape)
print(y_dev.shape)

In [ ]:
Xy_test = pd.concat([X_test, y_test], axis=1)
Xy_test = Xy_test[Xy_test["Stance"] != "unrelated"]
y_test = Xy_test["Stance"]
X_test = Xy_test.drop("Stance", axis=1)
print(X_test.shape)
print(y_test.shape)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_train)
print(classification_report(y_train, preds))

score, max_score = score_4class(y_train, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

In [ ]:
preds = clf.predict(X_dev)
print(classification_report(y_dev, preds))

score, max_score = score_4class(y_dev, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

In [ ]:
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

score, max_score = score_4class(y_test, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, preds, labels=["agree", "disagree", "discuss"], sample_weight=None)